In [ ]:
import gmsh
import os
import import_ipynb
import numpy as np
import matplotlib.pyplot as plt
from fem_pre_processing import read_mesh
from fem_processing import gaussian_quadrature, matrices_assembly
from fem_pos_processing import graph_results

# Project 1: Poisson Problem at $L$ Domain   

Considere a geometria L apresentada na Figura $(1)$.

<figure>
    <img src="pre_processing/pictures/L_domain.png" alt="Fig.1" style="width:20%;" />
    <figcaption>Figure 1: Geometry of L domain.</figcaption>
</figure>

Não há cargas no domínio e a solução exata para o problema é

$$
V(r, \theta) = r^{2/3} sin \left( \frac{2\theta}{3} + \frac{\pi}{3} \right)
\tag{1}
$$

onde $r$ e $\theta$ são as coordenadas polares, 

$$
\begin{cases}
r = \sqrt{x^2 + y^2}  \\
\theta = arctan(y/x) \tag{2}
\end{cases}
$$

A variação de $\theta$ para a geometria é $-\pi/2 \leq \theta \leq \pi$.

As condições de contorno são todas de Dirichlet e podem ser calculadas utilizando a expressão da solução exata sobre as fronteiras.

Usar malhas com densidades diferentes e verificar a convergência da sua solução à medida que refinna a malha. Calcule os erros na norma $L_2$ e na norma de energia. A melhor forma de verificar a convergência é traçar gráfios em escala _log-log_ da norma do erro em função de h. Se tudo estiver correto, em escala _log-log_ você obterá retas cujas inclinações fornecerão as taxas de convergência.

# `set_analytical_solution()`

In [ ]:
def set_analytical_solution(x, y):
    """
    Calcula a solução analítica u(x, y) e seu gradiente em coordenadas cartesianas.

    Parâmetros:
    - x, y: Coordenadas cartesianas do ponto.

    Retorna:
    - u: Valor da solução analítica no ponto (x, y).
    - grad_u: Vetor gradiente da solução analítica no ponto (x, y).
    """
    # Converter para coordenadas polares
    r = np.sqrt(x**2 + y**2)
    theta = np.arctan2(y, x)

    # Solução analítica
    u = r**(2/3) * np.sin(2 * theta / 3 + np.pi / 3)
    
    # Verificação para evitar problemas numéricos em R ~ 0
    epsilon = 1e-10
    if r < epsilon:
        return u, np.array([0.0, 0.0])  # Gradiente indefinido no ponto singular
    
    # Derivadas parciais em coordenadas polares
    du_dR = (2 / 3) * r**(-1/3) * np.sin(2 * theta / 3 + np.pi / 3)
    du_dtheta = r**(2/3) * (2 / 3) * np.cos(2 * theta / 3 + np.pi / 3)
    
    # Gradiente em coordenadas cartesianas
    grad_x = du_dR * (x / r) - (1 / r) * du_dtheta * (y / r)
    grad_y = du_dR * (y / r) + (1 / r) * du_dtheta * (x / r)
    grad_u = np.array([grad_x, grad_y])

    return u, grad_u

# `apply_physics()`

In [ ]:
def apply_physics(FINITE_ELEMENT, mesh_data):
    """
    Adiciona uma nova chave 'source' a cada dicionário em conn_data.
    
    Parâmetros:
    - mesh_data: Dicionário contendo os dados da malha.
    - element_type: Tuple (tipo do elemento, ordem).
    
    Retorna:
    - mesh_data: O dicionário atualizado com a chave 'source' em cada elemento de conn_data.
    """
    # Dictionary with all nodes in the mesh
    cell_data = mesh_data['cell']
    nodes_data = mesh_data['nodes']

    # Adicionar as propriedades do materiais ao dicionário da célula
    for key, cell in cell_data.items():
        # Constant material properties
        a = cell['material']['a_constant']
        
        # Adicionar as propriedades dos materiais ao dicionário da célula
        cell['stiffness_term'] = []
        cell['mass_term'] = []
        cell['source'] = []
        
        # Get the global coordinates of the nodes
        aie = [nodes_data[idx]['xg'] for idx in cell['conn']]

        # Get the Gauss points and weights
        gauss_points, _ = gaussian_quadrature.gauss_data(FINITE_ELEMENT)

        # Adicionar a nova chave 'material' ao dicionário da célula
        for xik in gauss_points: 
            xge, yge = matrices_assembly.isomapping_to_global_coordinates(FINITE_ELEMENT, aie, xik)
            cell['stiffness_term'].append(a * np.eye(2))
            cell['mass_term'].append(1)
            cell['source'].append(0)
            
        # Verifica tipo de fronteira absorvente (ABC)
        cell_data[key]['abc'] = {'type': None, 'conn_idx': None}
    
    # Adiciona os potenciais de Dirichlet
    for node in nodes_data.values():
        if node['bc']['type'] == 'Dirichlet':
            # Coordenadas cartesianas do nó
            x, y = node['xg']  

            # Atualizar o valor de Dirichlet para o nó
            node['bc']['value'] = set_analytical_solution(x, y)[0]

    return mesh_data

# `create_domain()`

In [ ]:
def create_domain(FINITE_ELEMENT, BOUNDARY, MATERIAL, h, auto_save=True, view_mesh=False):
    mesh_data = {}
    type, order = FINITE_ELEMENT
    vertices = [(0, 0, 0), (0, -1, 0), (1, -1, 0), (1, 1, 0), (-1, 1, 0), (-1, 0, 0)]

    # Inicializar o Gmsh
    gmsh.initialize()
    gmsh.model.add("L_domain")

    # Adicionar pontos ao modelo
    point_tags = []
    for vertex in vertices:
        # Refinar no ponto (0, 0)
        local_lc = h * 0.2 if vertex == (0, 0, 0) else h  
        tag = gmsh.model.geo.addPoint(*vertex, local_lc)
        point_tags.append(tag)

    # Criar as linhas conectando os pontos
    line_tags = [gmsh.model.geo.addLine(point_tags[i], point_tags[(i + 1) % len(point_tags)])
        for i in range(len(point_tags))]

    # Criar os loops e as superfícies
    gmsh.model.geo.addCurveLoop(line_tags, 1)
    free_space = gmsh.model.geo.addPlaneSurface([1])

    # Adicionar grupos físicos para Dim=1
    for i, tag in enumerate([line_tags]):
        gmsh.model.addPhysicalGroup(1, tags=tag, tag=BOUNDARY[i]['tag'], name=BOUNDARY[i]['name'])

    # Adicionar grupos físicos para Dim=2
    gmsh.model.addPhysicalGroup(2, [free_space], tag=MATERIAL[0]['tag'], name=MATERIAL[0]['name']) 

    # Gerar a malha 2D
    gmsh.model.geo.synchronize()
    gmsh.option.setNumber("Mesh.SaveAll", 1)
    gmsh.model.mesh.generate(2)
    gmsh.model.mesh.setOrder(order)

    if view_mesh:
        gmsh.fltk.run()

    if auto_save:
        os.makedirs("pre_processing/mesh", exist_ok=True)
        gmsh.write(f"pre_processing/mesh/L_domain_{type}{order}.msh")
        read_mesh.basic_info()

    # Create mesh Structure Data from gmsh
    mesh_data['cell'] = read_mesh.get_cell_data(MATERIAL)
    mesh_data['nodes'] = read_mesh.get_nodes_data(BOUNDARY)
    mesh_data['edges'] = read_mesh.get_edge_data()

    # Apply physics to the problem
    mesh_data = apply_physics(FINITE_ELEMENT, mesh_data)
    
    gmsh.finalize()

    return mesh_data

# `plot_analytical_solution()`

In [ ]:
def plot_analytical_solution(Npts=100):
    vertices = [(0, 0, 0), (0, -1, 0), (1, -1, 0), (1, 1, 0), (-1, 1, 0), (-1, 0, 0)]    
    x_coords = [v[0] for v in vertices]
    y_coords = [v[1] for v in vertices]
    x_min, x_max = min(x_coords), max(x_coords)
    y_min, y_max = min(y_coords), max(y_coords)

    # Criar uma grade de pontos (x, y) no domínio
    x = np.linspace(x_min, x_max, Npts)
    y = np.linspace(y_min, y_max, Npts)
    X, Y = np.meshgrid(x, y)

    # Máscara para o domínio em forma de L
    mask = np.zeros_like(X, dtype=bool)
    for i in range(Npts):
        for j in range(Npts):
            px, py = X[i, j], Y[i, j]
            # Verificar se o ponto (px, py) está dentro do domínio L
            if (px >= 0 or py >= 0):  # Parte direita ou parte superior
                mask[i, j] = True

    # Calcular o potencial analítico no domínio válido
    potential = np.full_like(X, np.nan, dtype=float)  # Preencher com NaN para fora do domínio
    for i in range(Npts):
        for j in range(Npts):
            if mask[i, j]:  
                # Solução analícica do Potencial
                potential[i, j] = set_analytical_solution(X[i, j], Y[i, j])[0]

    # Plotar o potencial
    plt.figure(figsize=(8, 6))
    contour = plt.contourf(X, Y, potential, levels=50, cmap='viridis')
    plt.colorbar(contour, label=r"$u(x, y)$")
    plt.xlabel(r'$x$')
    plt.ylabel(r'$y$')
    plt.axis("equal")
    plt.tight_layout()

    # Salvar a figura
    filepath = graph_results.get_dir(f"pos_processing/pictures/L_analytical_solution.svg")
    plt.savefig(filepath, format="svg")
    plt.close()
    print(f"Arquivo salvo em: {filepath}")

Conversão do arquivo Jupyter Notebook para um script Python: ``python -m nbconvert --to script name.ipynb``

Belo Horizonte, Brazil. 2024.  
Adilton Junio Ladeira Pereira - adt@ufmg.br  
&copy; All rights reserved.